In [1]:

!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-771b0a94-4262-8b78-b289-b6a51d146afd)


In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [4]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [7]:
!pip list keras

Package                       Version       
----------------------------- --------------
absl-py                       0.10.0        
alabaster                     0.7.12        
albumentations                0.1.12        
altair                        4.1.0         
appdirs                       1.4.4         
argon2-cffi                   20.1.0        
asgiref                       3.3.1         
astor                         0.8.1         
astropy                       4.2           
astunparse                    1.6.3         
async-generator               1.10          
atari-py                      0.2.6         
atomicwrites                  1.4.0         
attrs                         20.3.0        
audioread                     2.1.9         
autograd                      1.3           
Babel                         2.9.0         
backcall                      0.2.0         
backports.tempfile            1.0           
backports.weakref             1.0.post1     
beautifuls

In [5]:
"""지금까지 제일 잘 분류하는 신경망 코드 """

#import tensorflow as tf
from keras.models import *
from keras.layers import  Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Activation
from sklearn.model_selection import train_test_split
from keras.datasets.fashion_mnist import load_data
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import to_categorical
from keras.layers.normalization import BatchNormalization
import loader3
from PIL import Image



Using TensorFlow backend.


In [6]:


"""이미지 경로 설정 """

train_image = '/content/gdrive/MyDrive/samples6/train_resize2/'
test_image = '/content/gdrive/MyDrive/samples6/test_resize2/'
train_label = '/content/gdrive/MyDrive/samples6/train_label.csv'
test_label = '/content/gdrive/MyDrive/samples6/test_label.csv'



""" loader3 함수를 가상환경 밑에 넣고 함수 구현 """

#image = tf.keras.preprocessing.image.load_img(train_image)
#input_arr = keras.preprocessing.image.img_to_array(image)
#x_train = np.array([input_arr]) 

x_train = loader3.image_load(train_image)
y_train = loader3.label_load(train_label)

#image2 = tf.keras.preprocessing.image.load_img(test_image)
#input_arr2 = keras.preprocessing.image.img_to_array(image2)
#x_test = np.array([input_arr2]) 

x_test = loader3.image_load(test_image)
y_test = loader3.label_load(test_label)      


print ( loader3.image_load(train_image).shape )
print ( loader3.image_load(test_image).shape )   # (400,)
print ( loader3.label_load(train_label).shape)   # (1600, 2)
print ( loader3.label_load(test_label).shape )

"""4차원 데이터이므로 reshape 필요없음"""


## x_train, x_test 수치 정규화를 위한 작업

#x_mean = np.mean(x_train, axis = (0, 1, 2))
#x_std = np.std(x_train, axis = (0, 1, 2))

#x_train = (x_train - x_mean) / x_std
#x_test = (x_test - x_mean) / x_std

x_train = x_train.astype('float')
x_test = x_test.astype('float')
x_train = x_train / 255
x_test = x_test /255



(1600, 32, 32, 3)
(400, 32, 32, 3)
(1600, 2)
(400, 2)


In [7]:


# 검증 데이터셋을 만듭니다.

from sklearn.model_selection import train_test_split
np.random.seed(111)
# 훈련/테스트 데이터를 0.7/0.3의 비율로 분리합니다.

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,
                                                  test_size = 0.2, random_state = 777)


In [8]:


## 제너레이터 사용

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(horizontal_flip = True,
                                vertical_flip=True,
                                shear_range=0.5,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   rotation_range = 30,
                                   fill_mode = 'nearest')

val_datagen = ImageDataGenerator()

batch_size = 32

train_generator = train_datagen.flow(x_train, y_train,
                                    batch_size =32)

val_generator = val_datagen.flow(x_val, y_val,
                                batch_size =32)



from keras.models import *
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, BatchNormalization, Activation
from keras.optimizers import Adam
from keras.applications import VGG16


# imagenet을 학습한 모델을 불러옵니다.

vgg16 = VGG16(weights = 'imagenet', input_shape = (32, 32, 3), include_top = False)   


for layer in vgg16.layers[:-4]:
    layer.trainable = False
    

model = Sequential()    

model.add(vgg16)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(2, activation = 'softmax'))




Instructions for updating:
If using Keras pass *_constraint arguments to layers.

58892288/58889256 [==============================] - 1s 0us/step


In [9]:

from keras.callbacks import ModelCheckpoint, EarlyStopping

filepath = 're_w2.h5'

callbacks = [ ModelCheckpoint( filepath = filepath , monitor = 'val_loss' , verbose=1, save_best_only=True ) ]


model.compile(optimizer = Adam(1e-4),
             loss = 'binary_crossentropy',
             metrics = ['acc'])


batch_size=32


def get_step(train_len, batch_size):      
    if(train_len % batch_size > 0):
        return train_len // batch_size + 1
    else:
        return train_len // batch_size


history = model.fit(train_generator,
                    epochs = 30,
                    steps_per_epoch = get_step(len(x_train), batch_size),
                    validation_data = val_generator,
                    validation_steps = get_step(len(x_val), batch_size), callbacks=callbacks )

print( model.evaluate(x_test,y_test))

save_model(model, 're_w3.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/30
40/40 [==============================] - 4s 111ms/step - loss: 0.4942 - acc: 0.7555 - val_loss: 0.3827 - val_acc: 0.8656

Epoch 00001: val_loss improved from inf to 0.38273, saving model to re_w2.h5
Epoch 2/30
40/40 [==============================] - 1s 19ms/step - loss: 0.3662 - acc: 0.8375 - val_loss: 0.2245 - val_acc: 0.9094

Epoch 00002: val_loss improved from 0.38273 to 0.22454, saving model to re_w2.h5
Epoch 3/30
40/40 [==============================] - 1s 23ms/step - loss: 0.3582 - acc: 0.8469 - val_loss: 0.1895 - val_acc: 0.8969

Epoch 00003: val_loss improved from 0.22454 to 0.18953, saving model to re_w2.h5
Epoch 4/30
40/40 [==============================] - 1s 23ms/step - loss: 0.3141 - acc: 0.8680 - val_loss: 0.3234 - val_acc: 0.9031

Epoch 00004: val_loss did not improve from 0.18953
Epoch 5/30
40/40 [==============================] - 1s 24ms/step - loss: 0.2881 - acc: